In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import surprise as srp
import os

## download and clean data

In [ ]:
data = pd.read_table('/kaggle/input/movielens-100k-dataset/ml-100k/u1.base',header=None)

In [ ]:
data.head()

In [ ]:
files_dir = os.path.expanduser('/kaggle/input/movielens-100k-dataset/ml-100k/')

reader = srp.Reader('ml-100k')
train_file = files_dir + 'u%d.base'
test_file = files_dir + 'u%d.test'
folds_files = [(train_file % i, test_file % i) for i in (1, 2, 3, 4, 5)]

data = srp.Dataset.load_from_folds(folds_files, reader=reader)
pkf = srp.model_selection.PredefinedKFold()

algo = srp.SVD()
print('svd algorithm')
for trainset, testset in pkf.split(data):

    algo.fit(trainset)
    predictions = algo.test(testset)

    srp.accuracy.rmse(predictions, verbose=True)
    
    
algo = srp.SVDpp()
print('svdpp algorithm')
for trainset, testset in pkf.split(data):

    algo.fit(trainset)
    predictions = algo.test(testset)

    srp.accuracy.rmse(predictions, verbose=True)

---------

--------

In [ ]:
def mean(a):
    counter=0
    s=0
    for i in a:
        if i!=0:
            counter+=1
            s+=i
    return s/counter

In [ ]:
def rating_mean(G):
    s=0
    counter=0
    for i in G:
        for j in i:
            if j!=0:
                s+=j
                counter+=1
    return s/counter

In [ ]:
def optimism(G,index,G_mean):
    user_index=index
    s1=0
    s2=0
    for i in range(len(G[user_index])):
        if G[user_index][i] !=0 and mean(G[:,i])<=3:
            s2+=1
            if G[user_index][i]>3:
                s1+=1
    if s2==0:
        return 0
    else:
        return s1/s2

In [ ]:
def pessimism(G,index,G_mean):
    user_index=index
    s1=0
    s2=0
    for i in range(len(G[user_index])):
        if G[user_index][i] !=0 and mean(G[:,i])>=3:
            s2+=1
            if G[user_index][i]<3:
                s1+=1
    if s2==0:
        return 0
    else:
        return s1/s2

In [ ]:
def MSE(a):
    s=0
    counter=0
    for i in a:
        for j in i:
            if j!=0:
                s+=j**2
                counter+=1
    s=s/counter
    return s

In [ ]:
def A_maker(G,G_mean):
    ms=[]
    for i in range(len(G)):
        ms.append(mean(G[i]))
        
    A=np.zeros((m,1))
    p_m=np.mean(personality)
    for i in range(m):
        if personality[i]<=p_m and ms[i]>=G_mean:
            A[i]=1
        elif personality[i]>p_m and ms[i]<G_mean:
            A[i]=-1
        else:
            A[i]=0
    return A

In [ ]:
def SVD(R,U,V,W,A,X,Y,l1,l2,l3,iteration,alpha):
    mse=[]
    for i in range(iteration):
        G_p = U@V.T
        G_p_m=rating_mean(G_p)
        A=A_maker(G_p,G_p_m)
        ru = l1*U-(np.multiply(np.multiply(W,(R-U@V.T)),W))@V
        ru = ru+(l3*(X.T@V@U.T+((-1*Y).T))@A)[0,0]*A@(X.T@V)
        U = U-(alpha*ru)
        
        rv = l2*V-(np.multiply(np.multiply(W.T,(R.T-V@U.T)),W.T)@U)
        rv = rv + ((l3*A.T@(U@V.T@X-Y))[0,0])*X@(A.T@U)
        V = V-(alpha*rv)
        
        mse.append(MSE(np.multiply(W,R-(U@V.T))))
#         if i>3 and mse[-1]-mse[-3]<=0.0001:
#             print('converged!')
#             break
        print( 'iteration '+str(i+1),end='\r')
    return U, V,mse

In [ ]:
path='/kaggle/input/movielens-100k-dataset/ml-100k/'
for p in (1, 2, 3, 4, 5):
    data = pd.read_table(path+'u'+str(p)+'.base',header=None)
    ratings = np.zeros((943,1682),dtype=int)
    
    for i in range(data.shape[0]):
        ratings[data.iloc[i][0]-1][data.iloc[i][1]-1]=data.iloc[i][2]
    G=ratings
    
    m=G.shape[0]
    n=G.shape[1]
    k=2
    
    U = np.ones((m,k))
    V = np.ones((n,k))
    
    W=np.zeros((G.shape[0],G.shape[1]))
    for i in range(G.shape[0]):
        for j in range(G.shape[1]):
            if G[i][j]!=0:
                W[i][j]=1
                
    X=np.full((n,1),1/n)
    
    G_mean=rating_mean(G)

    Y=np.full((m,1),G_mean)

    l1=l2=0.05
    l3=1
    
    personality=[]
    for i in range(len(G)):
        personality.append(optimism(G,i,G_mean)-pessimism(G,i,G_mean))
    
    A=A_maker(G,G_mean)
    
    u,v,mse=SVD(G,U,V,W,A,X,Y,l1,l2,l3,50,0.0002)
    
    data_test = pd.read_table(path+'u'+str(p)+'.test',header=None)
    
    rmse=0
    rp=u@v.T
    for i in range(data_test.shape[0]):
        rmse+=(rp[data_test.iloc[i][0]-1][data_test.iloc[i][1]-1]-data_test.iloc[i][2])**2
    rmse = rmse/data_test.shape[0]
    rmse = np.sqrt(rmse)
    print('fold '+str(p),end=' ')
    print(rmse)